# 필요 라이브러리

In [ ]:
import time
import re
import random
import pandas as pd
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
from datetime import datetime

class OhouScraper:
    def __init__(self):
        self.base_url = "https://ohou.se/store/category?category_id=10210006"  # 의자 카테고리 URL
        self.results = []
        self.processed_links = set()  # 중복 상품 방지를 위한 세트
        self.max_products = 10000  # 최대 수집할 제품 수 (필요에 따라 조정)
        self.scroll_pause_time = 2  # 스크롤 후 대기 시간
        
        # Selenium 설정
        self.setup_driver()

    def setup_driver(self):
        # Chrome 옵션 설정
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # 화면 표시 없이 실행
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--disable-popup-blocking")
        
        # User-Agent 설정
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
        
        # WebDriver 설정 - webdriver_manager 없이 직접 Chrome 드라이버 사용
        try:
            # 기본 방식으로 Chrome 드라이버 실행 (시스템에 설치된 ChromeDriver 사용)
            self.driver = webdriver.Chrome(options=chrome_options)
        except Exception as e:
            print(f"Chrome 드라이버 초기화 중 오류: {e}")
            # 만약 Chrome 드라이버 경로를 직접 지정해야 하는 경우 아래 코드 사용
            # chromedriver_path = "/path/to/chromedriver"  # 크롬 드라이버 경로를 실제 경로로 변경
            # self.driver = webdriver.Chrome(service=Service(chromedriver_path), options=chrome_options)
            raise
            
        # self.driver.implicitly_wait(5)  # 암시적 대기 시간 설정

    def extract_product_info(self, product):
        try:
            # 링크 추출
            link_element = product.find_element(By.CSS_SELECTOR, "a")
            link = link_element.get_attribute("href") if link_element else ""
            
            # 이미 처리한 링크인지 확인
            if link in self.processed_links:
                return None
            self.processed_links.add(link)
            
            # 브랜드명 추출
            try:
                brand_element = product.find_element(By.CSS_SELECTOR, "div.product-brand")
                brand = brand_element.text.strip()
            except:
                brand = ""
            
            # 제품명 추출
            try:
                name_element = product.find_element(By.CSS_SELECTOR, "span.product-name")
                name = name_element.text.strip()
            except:
                name = ""
            
            # 가격 추출 - 할인율 추출
            try:
                discount_price_element = product.find_element(By.CSS_SELECTOR, "span.css-nqvy3g.e175igv96")
                discount_price_text = discount_price_element.text.strip()
                discount_price = re.sub(r'[^\d]', '', discount_price_text)  # 숫자만 남기기
            except:
                discount_price = "None"
                
            try:
            # 가격 추출 - 금액 추출
                price_element = product.find_element(By.CSS_SELECTOR, "span.css-13aof0h.e175igv95")
                price_text = price_element.text.strip()
                price = re.sub(r'[^\d]', '', price_text)  # 숫자만 남기기
            except:
                price = "None"
            
            # 별점 추출
            try:
                rating_element = product.find_element(By.CSS_SELECTOR, "div.review strong.avg") 
                rating = rating_element.text.strip()
            except:
                rating = "None"
            
            # 리뷰 개수 추출
            try:
                review_element = product.find_element(By.CSS_SELECTOR, "div.review span.count")
                review_text = review_element.text.strip()
                review_count = re.sub(r'[^\d]', '', review_text)  # 숫자만 남기기
            except:
                review_count = "0"
            
            return {
                "링크": link,
                "브랜드명": brand,
                "제품명": name,
                "할인율": discount_price,
                "가격": price,
                "별점": rating,
                "리뷰 갯수": review_count
            }
        except Exception as e:
            print(f"상품 정보 추출 중 오류 발생: {e}")
            return None

    def scroll_page(self):
        # 현재 스크롤 위치 저장
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        
        # 페이지 하단으로 스크롤
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 페이지 로딩 대기
        time.sleep(self.scroll_pause_time)
        
        # 새 스크롤 높이 계산
        new_height = self.driver.execute_script("return document.body.scrollHeight")
        
        # 스크롤이 더 이상 내려가지 않으면 False 반환
        return new_height != last_height

    def scrape_all_products(self):
        print("웹 페이지 로딩 중...")
        self.driver.get(self.base_url)
        time.sleep(3)  # 초기 페이지 로딩 대기
        
        no_new_products_count = 0
        initial_products_count = 0
        
        print("무한 스크롤을 통해 제품 수집 시작...")
        
        try:
            while len(self.results) < self.max_products:
                # 현재 상품 수집
                products = self.driver.find_elements(By.CSS_SELECTOR, "article.css-46f9vw.etj6rb20")
                
                if initial_products_count == 0:
                    initial_products_count = len(products)
                    print(f"초기 상품 수: {initial_products_count}")
                
                # 이전에 처리되지 않은 상품만 처리
                new_products_found = False
                for product in products:
                    try:
                        product_info = self.extract_product_info(product)
                        if product_info:
                            self.results.append(product_info)
                            new_products_found = True
                            print(f"상품 추출 ({len(self.results)}): {product_info['브랜드명']} - {product_info['제품명']}")
                    except StaleElementReferenceException:
                        # 요소가 DOM에서 제거됐을 때 발생하는 에러 무시
                        continue
                
                # 새 상품이 발견되지 않으면 카운터 증가
                if not new_products_found:
                    no_new_products_count += 1
                else:
                    no_new_products_count = 0
                
                # 5번 연속으로 새 상품이 발견되지 않으면 스크롤 중단
                if no_new_products_count >= 5:
                    print("5번 연속으로 새 상품이 발견되지 않아 스크롤 중단")
                    break
                
                # 스크롤 다운
                can_scroll_more = self.scroll_page()
                
                # 더 이상 스크롤할 수 없으면 중단
                if not can_scroll_more:
                    print("더 이상 스크롤할 수 없습니다. 모든 상품을 로드했습니다.")
                    break
                
                # 서버 부하 방지를 위한 추가 대기
                wait_time = random.uniform(1.0, 2.0)
                time.sleep(wait_time)
                
                print(f"현재까지 {len(self.results)}개 상품 수집 완료. 스크롤 계속...")
                
        except Exception as e:
            print(f"스크래핑 중 오류 발생: {e}")
        
        print(f"총 {len(self.results)}개 상품 수집 완료!")
        return self.results

    def save_to_excel(self, filename_prefix="오늘의집_의자"):
        now = datetime.now().strftime("%Y%m%d-%H%M")  # 현재 날짜/시간 문자열 생성
        filename = f"{filename_prefix}_{now}.xlsx"
        
        df = pd.DataFrame(self.results)
        df.to_excel(filename, index=False)
        
        print(f"결과가 {filename}에 저장되었습니다.")
        return df
        
    def close(self):
        """드라이버 종료"""
        if hasattr(self, 'driver') and self.driver:
            self.driver.quit()
            print("브라우저 드라이버가 종료되었습니다.")

def main():
    print("오늘의집 의자 제품 스크래퍼를 시작합니다...")
    scraper = OhouScraper()
    
    try:
        # 무한 스크롤을 통한 모든 제품 수집
        scraper.scrape_all_products()
        
        # Excel 형식으로도 저장
        scraper.save_to_excel()
        
    finally:
        # 브라우저 드라이버 종료 - 메모리 누수 방지
        scraper.close()

if __name__ == "__main__":
    main()



오늘의집 의자 제품 스크래퍼를 시작합니다...
웹 페이지 로딩 중...
무한 스크롤을 통해 제품 수집 시작...
초기 상품 수: 6
상품 추출 (1): 다니카 - 뮤즈 편한 팔걸이 컴퓨터 메쉬 사무용 책상 의자
상품 추출 (2): 다니카 - 뮤즈 편한 팔걸이 헤드형 컴퓨터 메쉬 책상 의자
상품 추출 (3): 네오체어 - [무료배송]버디시리즈(CPS/CPSH) 아마존 최다판매 학생용사무용 메쉬의자
상품 추출 (4): 일루일루 - 미니 타이탄 기본형 게이밍 학생 컴퓨터 사무용 발받침 의자 3color
상품 추출 (5): 린백토리 - 포커스 LTY100S 학생 바퀴없는 허리 편한 공부 책상 사무실 회의실의자
상품 추출 (6): 비애노 - 베어허그 리클라이너 컴퓨터 의자 책상
현재까지 6개 상품 수집 완료. 스크롤 계속...
상품 추출 (7): 비알프렌드 - 미드리 미드센츄리 인테리어 학생 사무용 회전 컴퓨터 의자 (3color)
상품 추출 (8): 시디즈 - 아이블 서울대 책상 의자 높이조절형
상품 추출 (9): 일루일루 - 타이탄 플러스 게이밍 컴퓨터 사무용 리클라이너 발받침 의자 3color
상품 추출 (10): os퍼니처 - OSP300 파스텔 컴퓨터 게이밍 사무용 사무실 학생 책상 인테리어 의자 3colors
상품 추출 (11): 체어팩토리 - 몬트 레트로 체어 팔걸이의자 바퀴 오피스 컴퓨터 사무실 의자 책상
상품 추출 (12): 시디즈 - 에가 책상 의자 (높이조절형)
상품 추출 (13): 스탠다드랩 - C7 서울대 의자 공부 학생 의자 컴퓨터 책상 의자 독서실 의자 능률 체어
상품 추출 (14): 시디즈 - 탭스퀘어 컴퓨터 책상 의자
상품 추출 (15): 영가구 - 리벳 사무용 회의용 푹신한 가죽 인테리어 오피스 의자 4colors
상품 추출 (16): 베스트리빙 - 인조가죽 미드센츄리 모던 카누 사무용 컴퓨터의자(대형) 6colors
상품 추출 (17): 소담갤러리 - 레토 임스 오피스 가죽 체어 미드센츄리 컴퓨터 책상 의자
상품 추출 (18): 핀즈 

In [1]:
import time
import re
import random
import pandas as pd
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from bs4 import BeautifulSoup
from datetime import datetime

class OhouScraper:
    def __init__(self):
        self.base_url = "https://ohou.se/store/category?category_id=10210006"  # 의자 카테고리 URL
        self.results = []
        self.processed_links = set()  # 중복 상품 방지를 위한 세트
        self.processed_products_count = 0  # 처리된 제품 수 추적
        self.max_products = 10000  # 최대 수집할 제품 수
        self.scroll_pause_time = 3  # 스크롤 후 대기 시간 증가
        self.max_retry_attempts = 5  # 최대 재시도 횟수
        
        # Selenium 설정
        self.setup_driver()

    def setup_driver(self):
        # Chrome 옵션 설정
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # 화면 표시 없이 실행
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--disable-popup-blocking")
        
        # User-Agent 설정 - 최신 버전으로 업데이트
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
        
        # WebDriver 설정
        try:
            self.driver = webdriver.Chrome(options=chrome_options)
            # self.driver.implicitly_wait(10)  # 암시적 대기 시간 증가
        except Exception as e:
            print(f"Chrome 드라이버 초기화 중 오류: {e}")
            raise

    def extract_product_info(self, product):
        try:
            # 링크 추출
            link_element = product.find_element(By.CSS_SELECTOR, "a")
            link = link_element.get_attribute("href") if link_element else ""
            
            # 이미 처리한 링크인지 확인
            if link in self.processed_links:
                return None
            self.processed_links.add(link)
            
            # 브랜드명 추출
            try:
                brand_element = product.find_element(By.CSS_SELECTOR, "div.product-brand")
                brand = brand_element.text.strip()
            except:
                brand = ""
            
            # 제품명 추출
            try:
                name_element = product.find_element(By.CSS_SELECTOR, "span.product-name")
                name = name_element.text.strip()
            except:
                name = ""
            
            # 가격 추출 - 할인율 추출
            try:
                discount_price_element = product.find_element(By.CSS_SELECTOR, "span.css-nqvy3g.e175igv96")
                discount_price_text = discount_price_element.text.strip()
                discount_price = re.sub(r'[^\d]', '', discount_price_text)  # 숫자만 남기기
            except:
                discount_price = "None"
                
            try:
                # 가격 추출 - 금액 추출
                price_element = product.find_element(By.CSS_SELECTOR, "span.css-13aof0h.e175igv95")
                price_text = price_element.text.strip()
                price = re.sub(r'[^\d]', '', price_text)  # 숫자만 남기기
            except:
                price = "None"
            
            # 별점 추출
            try:
                rating_element = product.find_element(By.CSS_SELECTOR, "div.review strong.avg") 
                rating = rating_element.text.strip()
            except:
                rating = "None"
            
            # 리뷰 개수 추출
            try:
                review_element = product.find_element(By.CSS_SELECTOR, "div.review span.count")
                review_text = review_element.text.strip()
                review_count = re.sub(r'[^\d]', '', review_text)  # 숫자만 남기기
            except:
                review_count = "0"
            
            return {
                "링크": link,
                "브랜드명": brand,
                "제품명": name,
                "할인율": discount_price,
                "가격": price,
                "별점": rating,
                "리뷰 갯수": review_count
            }
        except Exception as e:
            print(f"상품 정보 추출 중 오류 발생: {e}")
            return None

    def scroll_page(self):
        """페이지를 스크롤하고 새 콘텐츠가 로드되었는지 확인합니다."""
        # 현재 스크롤 위치 저장
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        
        # 페이지 하단으로 스크롤
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 페이지 로딩 대기 (시간 증가)
        time.sleep(self.scroll_pause_time)
        
        # 새 스크롤 높이 계산
        new_height = self.driver.execute_script("return document.body.scrollHeight")
        
        # 스크롤이 더 이상 내려가지 않으면 False 반환
        return new_height != last_height

    def force_scroll(self, times=3):
        """강제로 여러 번 스크롤하여 새 콘텐츠를 로드합니다."""
        for i in range(times):
            # 현재 보이는 화면 높이의 70%만큼씩 스크롤
            self.driver.execute_script("window.scrollBy(0, window.innerHeight * 0.7);")
            time.sleep(1)  # 짧은 대기

    def get_product_count_from_page(self):
        """페이지에서 총 제품 수를 추출하는 시도"""
        try:
            # 페이지 상단에 표시되는 총 제품 수 추출 시도
            # 실제 CSS 선택자는 웹사이트에 따라 다를 수 있음
            count_element = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.category-feed__header span.count"))
            )
            count_text = count_element.text.strip()
            total_count = int(re.sub(r'[^\d]', '', count_text))
            return total_count
        except (NoSuchElementException, TimeoutException, ValueError) as e:
            print(f"총 제품 수를 가져오는 데 실패했습니다: {e}")
            return None

    def restart_browser(self):
        """브라우저를 재시작하고 현재 페이지로 돌아갑니다."""
        print("브라우저 재시작 중...")
        current_url = self.driver.current_url
        self.driver.quit()
        time.sleep(2)
        self.setup_driver()
        self.driver.get(current_url)
        time.sleep(5)  # 페이지 로드 대기
        print("브라우저 재시작 완료")

    def scrape_with_pagination(self):
        """페이지네이션 방식으로 스크래핑을 시도합니다."""
        print("페이지네이션 방식으로 시도 중...")
        base_url = "https://ohou.se/store/category?category_id=10210006&page="
        page_num = 1
        max_page = 100  # 최대 페이지 수 (필요에 따라 조정)
        
        while page_num <= max_page and len(self.results) < self.max_products:
            current_url = f"{base_url}{page_num}"
            print(f"페이지 {page_num} 로드 중: {current_url}")
            
            try:
                self.driver.get(current_url)
                time.sleep(3)  # 페이지 로드 대기
                
                # 현재 페이지의 제품 추출
                products = self.driver.find_elements(By.CSS_SELECTOR, "article.css-46f9vw.etj6rb20")
                
                if not products:
                    print(f"페이지 {page_num}에서 제품을 찾을 수 없습니다. 페이지네이션이 끝났을 수 있습니다.")
                    break
                
                products_found = 0
                for product in products:
                    product_info = self.extract_product_info(product)
                    if product_info:
                        self.results.append(product_info)
                        products_found += 1
                
                print(f"페이지 {page_num}에서 {products_found}개 제품 추출 완료 (총 {len(self.results)}개)")
                
                # 다음 페이지로
                page_num += 1
                
                # 서버 부하 방지를 위한 대기
                wait_time = random.uniform(2.0, 4.0)
                time.sleep(wait_time)
                
            except Exception as e:
                print(f"페이지 {page_num} 처리 중 오류: {e}")
                if "ERR_TOO_MANY_REQUESTS" in str(e) or "429" in str(e):
                    print("요청이 너무 많습니다. 10분 대기 후 다시 시도합니다.")
                    time.sleep(600)  # 10분 대기
                else:
                    page_num += 1  # 다음 페이지로 진행
        
        return self.results

    def scrape_all_products(self):
        print("웹 페이지 로딩 중...")
        self.driver.get(self.base_url)
        time.sleep(5)  # 초기 페이지 로딩 대기 시간 증가
        
        # 총 제품 수 가져오기 시도
        total_products = self.get_product_count_from_page()
        if total_products:
            print(f"카테고리 내 총 제품 수: {total_products}개")
        
        no_new_products_count = 0
        retry_count = 0
        max_no_new_streak = 10  # 새 제품이 없는 최대 연속 횟수 증가
        
        print("무한 스크롤을 통해 제품 수집 시작...")
        
        try:
            last_products_count = 0
            last_progress_report = 0
            
            while len(self.results) < self.max_products:
                # 현재 스크롤 위치에서 상품 수집
                products = self.driver.find_elements(By.CSS_SELECTOR, "article.css-46f9vw.etj6rb20")
                
                if not products:
                    print("현재 페이지에서 제품을 찾을 수 없습니다.")
                    retry_count += 1
                    if retry_count >= self.max_retry_attempts:
                        print("최대 재시도 횟수에 도달했습니다.")
                        break
                    
                    print(f"강제 스크롤 시도 (시도 {retry_count}/{self.max_retry_attempts})...")
                    self.force_scroll(5)  # 강제 스크롤 시도
                    continue
                
                # 이전에 처리되지 않은 상품만 처리
                new_products_found = 0
                for product in products:
                    try:
                        product_info = self.extract_product_info(product)
                        if product_info:
                            self.results.append(product_info)
                            new_products_found += 1
                    except StaleElementReferenceException:
                        continue
                
                # 새 상품 발견 여부에 따른 처리
                if new_products_found == 0:
                    no_new_products_count += 1
                    print(f"새 상품이 발견되지 않았습니다. ({no_new_products_count}/{max_no_new_streak})")
                    
                    if no_new_products_count >= max_no_new_streak:
                        print(f"{max_no_new_streak}번 연속으로 새 상품이 발견되지 않아 접근 방식 변경")
                        
                        # 브라우저 재시작 시도
                        if retry_count < self.max_retry_attempts:
                            print("브라우저 재시작 후 다시 시도합니다.")
                            self.restart_browser()
                            retry_count += 1
                            no_new_products_count = 0
                            continue
                        else:
                            # 페이지네이션 방식으로 전환
                            print("무한 스크롤 방식으로 더 이상 진행이 어렵습니다. 페이지네이션 방식으로 전환합니다.")
                            self.scrape_with_pagination()
                            break
                else:
                    no_new_products_count = 0
                    retry_count = 0
                
                # 프로그레스 로그 (100개마다 보고)
                if len(self.results) - last_progress_report >= 100:
                    print(f"현재까지 {len(self.results)}개 상품 수집 완료 (+{len(self.results) - last_progress_report}개)")
                    last_progress_report = len(self.results)
                    
                    # 총 제품 수와 비교하여 진행률 표시
                    if total_products:
                        progress = min(100, round(len(self.results) / total_products * 100, 1))
                        print(f"진행률: {progress}% ({len(self.results)}/{total_products})")
                
                # 스크롤 다운 (여러 번의 작은 스크롤로 변경)
                for _ in range(3):  # 3번의 작은 스크롤
                    self.driver.execute_script("window.scrollBy(0, window.innerHeight * 0.5);")
                    time.sleep(1)
                
                # 랜덤 지연 추가
                wait_time = random.uniform(1.5, 3.0)
                time.sleep(wait_time)
                
        except Exception as e:
            print(f"스크래핑 중 오류 발생: {e}")
        
        print(f"총 {len(self.results)}개 상품 수집 완료!")
        
        # 모든 제품을 수집하지 못했을 경우 페이지네이션 방식으로 보완
        if total_products and len(self.results) < total_products * 0.8:  # 80% 미만으로 수집된 경우
            print(f"무한 스크롤 방식으로 충분한 제품을 수집하지 못했습니다. ({len(self.results)}/{total_products})")
            print("페이지네이션 방식으로 추가 수집을 시도합니다.")
            self.scrape_with_pagination()
        
        return self.results

    def save_to_excel(self, filename_prefix="오늘의집_의자"):
        now = datetime.now().strftime("%Y%m%d-%H%M")  # 현재 날짜/시간 문자열 생성
        filename = f"{filename_prefix}_{now}.xlsx"
        
        df = pd.DataFrame(self.results)
        
        # 중복 제거 (혹시 모를 중복 제거)
        if "링크" in df.columns:
            df.drop_duplicates(subset=["링크"], keep="first", inplace=True)
        
        # 데이터 정리
        for col in df.columns:
            if df[col].dtype == object:  # 문자열 컬럼인 경우
                df[col] = df[col].str.strip()  # 앞뒤 공백 제거
        
        # 숫자 컬럼 변환 시도
        for col in ["가격", "리뷰 갯수"]:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")
        
        # 결과 저장
        df.to_excel(filename, index=False)
        
        print(f"결과가 {filename}에 저장되었습니다.")
        print(f"총 {len(df)}개의 고유한 제품 데이터가 저장되었습니다.")
        return df
        
    def close(self):
        """드라이버 종료"""
        if hasattr(self, 'driver') and self.driver:
            self.driver.quit()
            print("브라우저 드라이버가 종료되었습니다.")

def main():
    print("오늘의집 의자 제품 스크래퍼를 시작합니다...")
    scraper = OhouScraper()
    
    try:
        # 무한 스크롤을 통한 모든 제품 수집
        scraper.scrape_all_products()
        
        # Excel 형식으로 저장
        scraper.save_to_excel()
        
    except KeyboardInterrupt:
        print("\n사용자에 의해 스크래핑이 중단되었습니다.")
        # 중간 결과 저장
        if len(scraper.results) > 0:
            print(f"중간 결과 {len(scraper.results)}개 저장 중...")
            scraper.save_to_excel(filename_prefix="오늘의집_의자_중간결과")
    finally:
        # 브라우저 드라이버 종료 - 메모리 누수 방지
        scraper.close()

if __name__ == "__main__":
    main()

오늘의집 의자 제품 스크래퍼를 시작합니다...
웹 페이지 로딩 중...
총 제품 수를 가져오는 데 실패했습니다: Message: 
Stacktrace:
0   chromedriver                        0x0000000100c8aa54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000100c82cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x00000001007ce864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100815410 cxxbridge1$string$len + 382736
4   chromedriver                        0x0000000100856480 cxxbridge1$string$len + 649088
5   chromedriver                        0x00000001008097ec cxxbridge1$string$len + 334572
6   chromedriver                        0x0000000100c4fccc cxxbridge1$str$ptr + 2562928
7   chromedriver                        0x0000000100c52f98 cxxbridge1$str$ptr + 2575932
8   chromedriver                        0x0000000100c302c4 cxxbridge1$str$ptr + 2433384
9   chromedriver                        0x0000000100c53810 cxxbridge1$str$ptr + 2578100
10  chromedriver            